In [1]:
from trend_attention.modeling import TrendAttentionConfig


config = TrendAttentionConfig(
    input_dim = 5, 
    input_activation_name="Identity",
    hidden_feature_dim = 10,
    hidden_activation_function_name="LeakyReLU",
    embedding_dim = 4, 
    seq_len = 5, 
    num_heads = 2,
    masked_attention = True,
    use_bias = False,
    attention_dropout = 0.25,
    feature_engineering_dropout = 0.25,
    n_layers = 2,
    task = "binary_classification"
)

config = TrendAttentionConfig.from_file("config.json")
config = TrendAttentionConfig.from_file("config.yaml")

In [2]:
from trend_attention.modeling import FeatureEmbeddings
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

inputs = torch.randn(2, 5, 5, dtype=torch.float32)  # Random normal values
input_embedding_layer = FeatureEmbeddings(config, device)
feature_embeddings = input_embedding_layer(inputs.to(device))
feature_embeddings

tensor([[[ 0.8493, -0.8450, -0.0000,  0.6085],
         [-0.1143,  1.1573, -0.3575,  0.2099],
         [-0.5850,  1.3032,  0.0000,  0.4509],
         [-0.3807, -0.8406,  0.0000, -0.2162],
         [ 2.4165,  0.2081, -0.0000,  0.0000]],

        [[-0.0000, -0.0000, -0.3561, -0.0000],
         [ 0.0000, -0.0000,  0.0000,  0.2389],
         [ 0.7280,  0.8960,  0.7372,  0.0000],
         [ 0.0000,  0.4555,  0.4286,  0.0744],
         [ 0.8647,  0.9109,  0.0000,  0.7217]]], device='mps:0',
       grad_fn=<MulBackward0>)

In [3]:
from trend_attention.modeling import PositionalEncoding

positional_embedding_layer = PositionalEncoding(config, device)

feature_embeddings = positional_embedding_layer(feature_embeddings)
feature_embeddings

tensor([[[ 7.9012e-01, -5.0442e-01,  2.6071e-01,  1.9591e+00],
         [ 1.8760e-01,  1.7995e+00,  7.9401e-01, -6.2176e-01],
         [-7.5184e-01,  4.0527e-01, -3.9123e-01,  8.7741e-01],
         [-4.2570e-01, -2.5420e+00,  1.4469e+00,  4.3443e-01],
         [ 7.7836e-01, -2.0376e-01, -1.6989e-02,  6.6905e-01]],

        [[-5.9197e-02,  3.4055e-01, -9.5391e-02,  1.3506e+00],
         [ 3.0195e-01,  6.4221e-01,  1.1515e+00, -5.9279e-01],
         [ 5.6115e-01, -1.9802e-03,  3.4596e-01,  4.2647e-01],
         [-4.4976e-02, -1.2459e+00,  1.8755e+00,  7.2507e-01],
         [-7.7345e-01,  4.9912e-01, -1.6989e-02,  1.3908e+00]]],
       device='mps:0', grad_fn=<AddBackward0>)

In [5]:
from trend_attention.modeling import FeatureMultiHeadAttention, FeatureAttentionOutput, FeatureEngineering

attention = FeatureMultiHeadAttention(config, device)
attention_weights, attention_embeddings = attention(feature_embeddings)

output = FeatureAttentionOutput(config, device)
feature_embeddings = output(attention_embeddings, feature_embeddings)

engineering = FeatureEngineering(config, device)
feature_embeddings = engineering(feature_embeddings)
feature_embeddings

tensor([[[ 0.1327, -0.3486,  0.3546, -0.3382],
         [ 0.0595, -0.1937,  0.2785, -0.0122],
         [ 0.0895, -0.2006,  0.3734, -0.0093],
         [ 0.0159, -0.1696,  0.1494,  0.1006],
         [-0.0088,  0.0825,  0.0551, -0.2253]],

        [[-0.0230, -0.1794, -0.0967,  0.1408],
         [ 0.2330, -0.0682, -0.2800, -0.1163],
         [-0.0301, -0.1263, -0.1428, -0.3861],
         [-0.0417, -0.0959, -0.0703,  0.1568],
         [ 0.0175, -0.0663,  0.0741, -0.0761]]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [6]:
attention_weights

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4986, 0.5014, 0.0000, 0.0000, 0.0000],
          [0.3264, 0.3334, 0.3402, 0.0000, 0.0000],
          [0.2482, 0.2501, 0.2506, 0.2511, 0.0000],
          [0.1990, 0.2009, 0.2011, 0.2019, 0.1971]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4993, 0.5007, 0.0000, 0.0000, 0.0000],
          [0.3318, 0.3366, 0.3316, 0.0000, 0.0000],
          [0.2491, 0.2498, 0.2513, 0.2498, 0.0000],
          [0.1987, 0.2003, 0.2008, 0.2000, 0.2001]]],


        [[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4961, 0.5039, 0.0000, 0.0000, 0.0000],
          [0.3324, 0.3338, 0.3338, 0.0000, 0.0000],
          [0.2503, 0.2497, 0.2492, 0.2508, 0.0000],
          [0.2010, 0.1945, 0.1948, 0.2035, 0.2063]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5014, 0.4986, 0.0000, 0.0000, 0.0000],
          [0.3332, 0.3335, 0.3333, 0.0000, 0.0000],
          [0.2505, 0.2493, 0.2501, 0.2500, 0.0000],
    

In [9]:
from trend_attention.modeling import FeatureEngineeringAttentionLayer

inputs = torch.randn(2, 5, 5, dtype=torch.float32)  
input_embedding_layer = FeatureEmbeddings(config, device)
feature_embeddings = input_embedding_layer(inputs.to(device))
positional_embedding_layer = PositionalEncoding(config, device)
feature_embeddings = positional_embedding_layer(feature_embeddings)
feature_engineering_attention_layer = FeatureEngineeringAttentionLayer(config, device)
attention_weights, feature_embeddings = feature_engineering_attention_layer(feature_embeddings)
feature_embeddings

tensor([[[ 0.3565,  0.1209,  0.2129, -0.3026],
         [ 0.3851,  0.1071,  0.4626,  0.1007],
         [-0.0122, -0.1639, -0.1113, -0.3032],
         [ 0.0265, -0.0338, -0.1477, -0.6267],
         [-0.0779, -0.1147, -0.1510, -0.5834]],

        [[ 0.0936, -0.0207,  0.1795,  0.0798],
         [-0.0527, -0.2552,  0.0149, -0.2671],
         [-0.2831, -0.1184, -0.1829, -0.3321],
         [ 0.1757,  0.1229, -0.0007, -0.4460],
         [ 0.0034, -0.1179,  0.1718,  0.0585]]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [10]:
attention_weights

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4798, 0.5202, 0.0000, 0.0000, 0.0000],
          [0.3808, 0.3410, 0.2782, 0.0000, 0.0000],
          [0.2637, 0.2548, 0.2351, 0.2464, 0.0000],
          [0.1957, 0.1973, 0.2001, 0.1984, 0.2086]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5390, 0.4610, 0.0000, 0.0000, 0.0000],
          [0.4023, 0.2036, 0.3942, 0.0000, 0.0000],
          [0.2476, 0.2364, 0.2647, 0.2514, 0.0000],
          [0.2011, 0.1438, 0.2258, 0.2096, 0.2197]]],


        [[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5150, 0.4850, 0.0000, 0.0000, 0.0000],
          [0.3561, 0.3264, 0.3175, 0.0000, 0.0000],
          [0.2059, 0.2852, 0.3030, 0.2059, 0.0000],
          [0.1698, 0.2220, 0.2264, 0.1701, 0.2117]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5492, 0.4508, 0.0000, 0.0000, 0.0000],
          [0.3973, 0.3030, 0.2998, 0.0000, 0.0000],
          [0.2522, 0.2111, 0.2139, 0.3228, 0.0000],
    

In [12]:
from trend_attention.modeling import TrendAttentionHead

head = TrendAttentionHead(config, device)
logits = head(feature_embeddings)
logits

tensor([[0.3862],
        [0.0760]], device='mps:0', grad_fn=<LinearBackward0>)

tensor([1, 0], dtype=torch.int32)

In [2]:
from trend_attention.modeling import TrendAttentionClassifier
from trend_attention.modeling import TrendAttentionConfig
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

inputs = torch.randn(2, 5, 5, dtype=torch.float32)  # Random normal values
labels = torch.tensor([1, 0], dtype = torch.int32)

config = TrendAttentionConfig.from_file("config.yaml")
model = TrendAttentionClassifier(config, device)
outputs = model(inputs.to(device), labels.to(device))
outputs.attention_weights

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4998, 0.5002, 0.0000, 0.0000, 0.0000],
          [0.3315, 0.3336, 0.3349, 0.0000, 0.0000],
          [0.2490, 0.2474, 0.2489, 0.2547, 0.0000],
          [0.1968, 0.1967, 0.1984, 0.2041, 0.2040]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4999, 0.5001, 0.0000, 0.0000, 0.0000],
          [0.3312, 0.3296, 0.3392, 0.0000, 0.0000],
          [0.2507, 0.2495, 0.2529, 0.2468, 0.0000],
          [0.1953, 0.1937, 0.2112, 0.1949, 0.2049]]],


        [[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4949, 0.5051, 0.0000, 0.0000, 0.0000],
          [0.3349, 0.3340, 0.3311, 0.0000, 0.0000],
          [0.2523, 0.2540, 0.2472, 0.2465, 0.0000],
          [0.2005, 0.1998, 0.1999, 0.2007, 0.1991]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5011, 0.4989, 0.0000, 0.0000, 0.0000],
          [0.3303, 0.3257, 0.3440, 0.0000, 0.0000],
          [0.2502, 0.2482, 0.2534, 0.2482, 0.0000],
    

In [21]:
outputs.feature_embeddings

tensor([[[-0.0054, -0.2584, -0.2768, -0.0237],
         [ 0.1191, -0.3981,  0.1071,  0.1557],
         [-0.0013, -0.2419, -0.5851, -0.0307],
         [ 0.1380, -0.1755, -0.0965, -0.0715],
         [ 0.0561, -0.0282, -0.0702, -0.0659]],

        [[ 0.1808, -0.1314, -0.1843,  0.3589],
         [-0.0893, -0.3531, -0.0074,  0.0849],
         [ 0.1544, -0.1239, -0.5481, -0.5214],
         [ 0.2457,  0.0391, -0.3352, -0.5862],
         [ 0.2278, -0.1994, -0.3090, -0.0781]]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [22]:
outputs.logits

tensor([[-0.4362],
        [-0.4250]], device='mps:0', grad_fn=<LinearBackward0>)

In [23]:
outputs.loss

tensor(0.7190, device='mps:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [4]:
model.save_model("trained_model")

Model and config saved in: trained_model


In [5]:
loaded_model = TrendAttentionClassifier.load_model("trained_model", device=device)

Model successfully loaded from trained_model


In [25]:
from trend_attention.modeling import TrendAttentionClassifier
from trend_attention.modeling import TrendAttentionConfig
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

inputs = torch.randn(2, 5, 5, dtype=torch.float32)  # Random normal values
labels = torch.tensor([1.5, -0.5], dtype = torch.int32)

config = TrendAttentionConfig.from_file("config.yaml")
config.task = "regression"
model = TrendAttentionClassifier(config, device)
outputs = model(inputs.to(device), labels.to(device))
outputs.attention_weights

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4992, 0.5008, 0.0000, 0.0000, 0.0000],
          [0.3324, 0.3334, 0.3342, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
          [0.2003, 0.2001, 0.1996, 0.1999, 0.2001]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5003, 0.4997, 0.0000, 0.0000, 0.0000],
          [0.3336, 0.3331, 0.3332, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
          [0.1997, 0.2000, 0.2000, 0.2001, 0.2002]]],


        [[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5003, 0.4997, 0.0000, 0.0000, 0.0000],
          [0.3338, 0.3325, 0.3337, 0.0000, 0.0000],
          [0.2499, 0.2500, 0.2501, 0.2499, 0.0000],
          [0.2000, 0.1993, 0.1986, 0.2000, 0.2021]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5004, 0.4996, 0.0000, 0.0000, 0.0000],
          [0.3338, 0.3330, 0.3332, 0.0000, 0.0000],
          [0.2501, 0.2502, 0.2499, 0.2498, 0.0000],
    

In [26]:
outputs.feature_embeddings

tensor([[[-0.0165, -0.1406, -0.0276, -0.2615],
         [ 0.1723,  0.0704,  0.1253, -0.0147],
         [ 0.0717, -0.4009, -0.0253, -0.4406],
         [-0.0411, -0.1538, -0.0274, -0.1370],
         [-0.2033,  0.2783, -0.1121,  0.0762]],

        [[-0.0161,  0.5966,  0.1880,  0.0695],
         [ 0.0850, -0.3234,  0.0259, -0.3399],
         [-0.0403, -0.1860, -0.0299, -0.1596],
         [-0.3594,  0.1900, -0.1223,  0.0987],
         [ 0.0951, -0.0538,  0.0918, -0.1781]]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [27]:
outputs.logits

tensor([[-0.0135],
        [-0.2910]], device='mps:0', grad_fn=<LinearBackward0>)

In [28]:
outputs.loss

tensor(0.5559, device='mps:0', grad_fn=<MseLossBackward0>)